Installation de la librairie Python openai. C'est le SDK officiel d'OpenAI, mais il marche aussi avec Albert parce qu'Albert est compatible OpenAI (mêmes endpoints, même format).

In [14]:
!pip install openai -q

On importe la classe OpenAI depuis la librairie.
On crée un "client" = un objet qui sait parler à l'API Albert

base_url -> au lieu d'appeler les serveurs OpenAI (api.openai.com), on pointe vers les serveurs d'Etalab


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://albert.api.etalab.gouv.fr/v1",
    api_key=os.getenv("ALBERT_API_KEY", "ta-cle-ici")
)

print("✅ Client configuré")

✅ Client configuré


On liste les modèles disponible

In [16]:
models = client.models.list()

for model in models.data:
    print(f"📦 {model.id} → {model.type}")

📦 meta-llama/Llama-3.1-8B-Instruct → text-generation
📦 openai/whisper-large-v3 → automatic-speech-recognition
📦 Qwen/Qwen2.5-Coder-32B-Instruct-AWQ → text-generation
📦 openai/gpt-oss-120b → text-generation
📦 Qwen/Qwen3-Coder-30B-A3B-Instruct → text-generation
📦 mistralai/Ministral-3-8B-Instruct-2512 → image-text-to-text
📦 BAAI/bge-m3 → text-embeddings-inference
📦 BAAI/bge-reranker-v2-m3 → text-classification
📦 mistralai/Mistral-Small-3.2-24B-Instruct-2506 → image-text-to-text
📦 voxtral-small-2507 → automatic-speech-recognition


Génération de texte : "c'est quoi data.gouv.fr en une phrase ?" avec llama 3.1

In [17]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "C'est quoi data.gouv.fr en une phrase ?"}
    ]
)

print(response.choices[0].message.content)

Data.gouv.fr est un portail de données ouvertes de l'État français, qui permet de partager et de consulter les données publiques de manière libre et transparente.


Génération de texte : "quelles sont les missions de la DINUM et de ETALAB ?" avec Mistral Small 3.2

In [18]:
response = client.chat.completions.create(
    model="mistralai/Mistral-Small-3.2-24B-Instruct-2506",
    messages=[
        {"role": "user", "content": "quelles sont les missions de la DINUM et de ETALAB ?"}
    ]
)

print(response.choices[0].message.content)

La **DINUM** (*Direction Interministérielle du Numérique*) et **ETALAB** (*Etat Laboratory d’Innovation Publique*) sont deux entités françaises clés dans la transformation numérique de l'État. Voici leurs principales missions :

### **Missions de la DINUM**
La DINUM, créée en 2019, supervise la stratégie numérique de l'État. Ses missions principales sont :
1. **Pilotage de la transformation numérique** : Coordonner la modernisation des services publics numériques.
2. **Développement des plateformes interministérielles** : Gérer des outils comme **FranceConnect**, **API.gouv.fr** ou **data.gouv.fr**.
3. **Cybersécurité et souveraineté numérique** : Renforcer la sécurité des systèmes publics.
4. **Accompagnement des administrations** : Former les agents publics aux pratiques numériques.
5. **Promotion de l’open source** : Encourager l’usage de logiciels libres dans la sphère publique.

### **Missions d’ETALAB**
ETALAB, intégré à la DINUM, est spécialisé dans l’innovation ouverte et la da

Génération de text avec un system prompt :
*   **system** → contexte/instructions pour l'IA
*   **user** → ma question

In [19]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "Tu es un expert de l'administration française. Réponds de façon concise."},
        {"role": "user", "content": "Quelles sont les missions de la DINUM et d'Etalab ?"}
    ]
)

print(response.choices[0].message.content)

La DINUM (Direction interministérielle du numérique) et Etalab sont deux entités françaises chargées de promouvoir l'utilisation de la technologie numérique dans l'administration française.

- La DINUM a pour missions :
  - Développer la conception d'une administration numérique de qualité,
  - Coordonner les politiques numériques,
  - Favoriser l'innovation et la transformation numérique.

- Etalab a pour missions :
  + Proposer des outils numériques et des données ouvertes pour faciliter la vie des citoyens,
  + Favoriser l'innovation et le développement de start-up,
  + Contribuer à la transparence et à la responsabilité de l'administration française.


In [20]:
response = client.chat.completions.create(
    model="mistralai/Mistral-Small-3.2-24B-Instruct-2506",
    messages=[
        {"role": "system", "content": "Tu es un expert de l'administration française. Réponds de façon concise."},
        {"role": "user", "content": "Quelles sont les missions de la DINUM et d'Etalab ?"}
    ]
)

print(response.choices[0].message.content)

La DINUM (Direction Interministérielle du Numérique) a pour missions :

1. Pilotage de la transformation numérique de l'État.
2. Définition et mise en œuvre de la politique interministérielle du numérique.
3. Coordination des projets numériques entre les ministères.
4. Animation de la communauté des acteurs du numérique public.

Etalab, qui dépend de la DINUM, a pour missions :

1. Ouverture des données publiques (open data).
2. Promotion de la transparence et de la participation citoyenne.
3. Développement de services numériques innovants pour les citoyens.
4. Animation de la communauté des acteurs de l'open data et des données publiques.


Embedding

In [21]:
response = client.embeddings.create(
    model="BAAI/bge-m3",
    input="La DINUM pilote la transformation numérique de l'État.",
    encoding_format="float"
)

vector = response.data[0].embedding

print(f"Dimension du vecteur : {len(vector)}")
print(f"Premiers nombres : {vector[:5]}")

Dimension du vecteur : 1024
Premiers nombres : [-0.03750269, 0.0025921802, -0.03416456, -0.060384743, -0.05430524]


In [22]:
import numpy as np

def similarite(texte1, texte2):
    """Calcule la similarité entre deux textes (0 à 1)."""
    r1 = client.embeddings.create(model="BAAI/bge-m3", input=texte1, encoding_format="float")
    r2 = client.embeddings.create(model="BAAI/bge-m3", input=texte2, encoding_format="float")

    v1 = np.array(r1.data[0].embedding)
    v2 = np.array(r2.data[0].embedding)

    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Test
print(similarite(
    "La DINUM pilote le numérique de l'État",
    "La direction interministérielle coordonne la transformation digitale"
))

print(similarite(
    "La DINUM pilote le numérique de l'État",
    "Il fait beau aujourd'hui à Paris"
))

0.5405140925287717
0.28068769973901425


Mini RAG

1/ Création de la base documentaire

In [24]:
documents = [
    "data.gouv.fr est la plateforme des données ouvertes de l'État français.",
    "Etalab est un département de la DINUM qui coordonne la politique de la donnée.",
    "Albert est l'IA souveraine de l'État, hébergée en France.",
    "Le MCP (Model Context Protocol) permet aux IA d'accéder à des données externes.",
    "La DINUM pilote la transformation numérique de l'administration."
]

print(f"📚 Base de {len(documents)} documents")

📚 Base de 5 documents


2/ Indexation des documents (calcul des embeddings de tous les docs en une seule fois)

In [25]:
docs_embeddings = []

for doc in documents:
    r = client.embeddings.create(model="BAAI/bge-m3", input=doc, encoding_format="float")
    docs_embeddings.append(r.data[0].embedding)

print(f"✅ {len(docs_embeddings)} documents indexés")

✅ 5 documents indexés


3/ Recherche

In [26]:
def rechercher(question, top_k=2):
    """Trouve les documents les plus proches de la question."""
    # Embedding de la question
    r = client.embeddings.create(model="BAAI/bge-m3", input=question, encoding_format="float")
    q_vec = np.array(r.data[0].embedding)

    # Calcul des similarités avec chaque document
    scores = []
    for i, doc_emb in enumerate(docs_embeddings):
        doc_vec = np.array(doc_emb)
        score = np.dot(q_vec, doc_vec) / (np.linalg.norm(q_vec) * np.linalg.norm(doc_vec))
        scores.append((score, documents[i]))

    # Tri par score décroissant
    scores.sort(reverse=True)

    return scores[:top_k]

# Test
resultats = rechercher("C'est quoi Albert ?")

for score, doc in resultats:
    print(f"{score:.2f} → {doc}")

0.61 → Albert est l'IA souveraine de l'État, hébergée en France.
0.30 → Etalab est un département de la DINUM qui coordonne la politique de la donnée.


In [27]:
resultats = rechercher("C'est quoi Albert ?", top_k=5)

for score, doc in resultats:
    print(f"{score:.2f} → {doc}")

0.61 → Albert est l'IA souveraine de l'État, hébergée en France.
0.30 → Etalab est un département de la DINUM qui coordonne la politique de la donnée.
0.28 → data.gouv.fr est la plateforme des données ouvertes de l'État français.
0.27 → Le MCP (Model Context Protocol) permet aux IA d'accéder à des données externes.
0.26 → La DINUM pilote la transformation numérique de l'administration.


In [29]:
def rag(question):
    """Répond à une question en utilisant les documents pertinents."""
    # 1. Recherche des documents pertinents
    resultats = rechercher(question, top_k=2)

    # 2. Construction du contexte
    contexte = "\n".join([doc for score, doc in resultats])

    # 3. Génération de la réponse
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": f"Réponds en te basant uniquement sur ce contexte :\n{contexte}"},
            {"role": "user", "content": question}
        ]
    )

    return response.choices[0].message.content

print(rag("Qui a développé Albert ?"))

Je n'ai pas trouvé d'informations précises sur la question spécifique du développement d'Albert. Cependant, on sait que la DINUM (Direction Interministérielle à l'Intelligence Artificielle) pilote la transformation numérique de l'administration et que Albert est l'IA souveraine de l'État, hébergée en France. 

Il semblerait que le développement d'Albert soit lié à la stratégie numérique de l'État français, mais sans plus d'informations précises, je ne peux pas identifier avec certitude les entités impliquées dans ce développement.


In [30]:
def rag(question):
    """Répond à une question en utilisant les documents pertinents."""
    # 1. Recherche des documents pertinents
    resultats = rechercher(question, top_k=2)

    # 2. Construction du contexte
    contexte = "\n".join([doc for score, doc in resultats])

    # 3. Génération de la réponse
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": f"Réponds en te basant uniquement sur ce contexte :\n{contexte}"},
            {"role": "user", "content": question}
        ]
    )

    return response.choices[0].message.content

# Test
print(rag("C'est quoi Albert ?"))

Albert est l'IA souveraine de l'État français, c'est-à-dire une intelligence artificielle développée et déployée par l'État français pour répondre à ses besoins en matière de données et d'analyse. Il s'agit d'une plateforme de traitement et d'analyse de données qui peut être utilisée pour améliorer la prise de décision publique.

Comme son nom l'indique, Albert est une intelligence artificielle souveraine, ce qui signifie qu'elle est développée et contrôlée par l'État français, et non pas par une entreprise privée. Cela permet d'assurer la sécurité et la confidentialité des données traitées par la plateforme.

Albert est hébergé en France et est conçu pour répondre aux besoins des administrations publiques françaises en matière de données et d'analyse. Il peut être utilisé pour analyser des données de diverses sources, telles que des données administratives, des données de santé ou des données économiques, pour identifier des tendances, des modèles et des opportunités de croissance.

E

In [31]:
def rag_strict(question):
    resultats = rechercher(question, top_k=2)
    contexte = "\n".join([doc for score, doc in resultats])

    response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": f"""Réponds UNIQUEMENT avec les informations du contexte ci-dessous.
Si l'information n'est pas dans le contexte, dis "Je n'ai pas cette information."
Sois concis, 2-3 phrases maximum.

Contexte :
{contexte}"""},
            {"role": "user", "content": question}
        ]
    )

    return response.choices[0].message.content

print(rag_strict("C'est quoi Albert ?"))

Albert est l'IA souveraine de l'État, hébergée en France.


le renforcement du prompt système entre les 2 permets d'avoir une réponse plus fidèle au contexte